In [27]:
import pandas as pd
import requests

from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [4]:
num = 5000
token = '53a4c7b8-6dab-452e-96cd-fcb8e146ebb3'

def data_gen(num):
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest?limit={}'.format(num)
    headers={'X-CMC_PRO_API_KEY': token}
    key_of_interest = (
        'id', 
        'cmc_rank', 
        'date_added', 
        'max_supply', 
        'name', 
        'symbol', 
        'slug', 
        'tags', 
        'circulating_supply'
    )
    res = requests.get(url, headers=headers).json()
    for c in res.get('data'):
        yield {
            k:v for k,v in c.items() if k in key_of_interest
        }    

In [5]:
df = pd.DataFrame(data_gen(5000))

In [15]:
te = TransactionEncoder()
tags = te.fit_transform(df.tags)
tags = pd.DataFrame(tags, columns=te.columns_)

In [46]:
freq_itemset = apriori(tags, min_support=0.01, use_colnames=True)
freq_itemset['itemsets_length'] = freq_itemset['itemsets'].apply(lambda x: len(x))


In [47]:
freq_itemset

,support,itemsets,itemsets_length
0,0.010276,(ai-big-data),1
1,0.011500,(asset-management),1
2,0.014681,(collectibles-nfts),1
3,0.010766,(content-creation),1
4,0.016883,(dao),1
5,0.043308,(defi),1
6,0.011011,(distributed-computing),1
7,0.010276,(dot-ecosystem),1
8,0.016883,(enterprise-solutions),1
9,0.015904,(ethereum),1


In [36]:
association_rules(
    fpgrowth(tags, min_support=0.01, use_colnames=True),
    metric='support',
    min_threshold=0.0005,
)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(mineable),(pow),0.114020,0.030585,0.027404,0.240343,7.858266,0.023917,1.276123
1,(pow),(mineable),0.030585,0.114020,0.027404,0.896000,7.858266,0.023917,8.519038
2,(scrypt),(mineable),0.021042,0.114020,0.014681,0.697674,6.118874,0.012281,2.930549
3,(mineable),(scrypt),0.114020,0.021042,0.014681,0.128755,6.118874,0.012281,1.123631
4,(scrypt),(pow),0.021042,0.030585,0.011011,0.523256,17.108372,0.010367,2.033408
5,(pow),(scrypt),0.030585,0.021042,0.011011,0.360000,17.108372,0.010367,1.529621
6,(mineable),(hybrid-pow-pos),0.114020,0.020798,0.013947,0.122318,5.881318,0.011575,1.115668
7,(hybrid-pow-pos),(mineable),0.020798,0.114020,0.013947,0.670588,5.881318,0.011575,2.689582


In [43]:
from pprint import pformat
print(pformat(list(tags.columns)))

['adult',
 'agriculture',
 'ai-big-data',
 'amm',
 'argon2',
 'art',
 'asset-management',
 'atomic-swaps',
 'binance-chain',
 'binance-launchpool',
 'binance-smart-chain',
 'blake',
 'blake256',
 'blake2b',
 'blake2s',
 'centralized-exchange',
 'coinbase-ventures-portfolio',
 'collectibles-nfts',
 'commodities',
 'content-creation',
 'cosmos',
 'crowdfunding',
 'crowdsourcing',
 'cryptonight',
 'cryptonight-lite',
 'cuckoo-cycle',
 'cybersecurity',
 'dag',
 'dagger-hashimoto',
 'dao',
 'dapp',
 'data-availability-proof',
 'data-provenance',
 'decentralized-exchange',
 'defi',
 'derivatives',
 'discount-token',
 'distributed-computing',
 'dot-ecosystem',
 'dpor',
 'dpos',
 'e-commerce',
 'education',
 'energy',
 'enterprise-solutions',
 'entertainment',
 'equihash',
 'escrow',
 'ethash',
 'ethereum',
 'events',
 'fan-token',
 'filesharing',
 'food-beverage',
 'gambling',
 'gaming',
 'geospatial-services',
 'groestl',
 'hardware',
 'health',
 'hospitality',
 'hybrid-dpow-pow',
 'hybrid-p

In [57]:
apriori(tags[tags['collectibles-nfts']], min_support=0.05, use_colnames=True).sort_values('support', ascending=False)

,support,itemsets
0,1.000000,(collectibles-nfts)
3,0.133333,(gaming)
14,0.133333,"(yield-farming, collectibles-nfts)"
7,0.133333,(yield-farming)
10,0.133333,"(gaming, collectibles-nfts)"
2,0.100000,(dao)
6,0.100000,(platform)
13,0.100000,"(platform, collectibles-nfts)"
9,0.100000,"(dao, collectibles-nfts)"
12,0.050000,"(payments, collectibles-nfts)"
